In [1]:
from itertools import product
import numpy as np
import fastplotlib as fpl
import imageio.v3 as iio

fastplotlib version from git (0.3.0) and __version__ (0.5.0) don't match.
rendercanvas version from git (2.0.0) and __version__ (2.0.3) don't match.
Unable to find extension: VK_EXT_swapchain_colorspace


Image(value=b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x01,\x00\x00\x007\x08\x06\x00\x00\x00\xb6\x1bw\x99\x…

Valid,Device,Type,Backend,Driver
✅ (default),AMD RADV POLARIS10 (ACO),DiscreteGPU,Vulkan,Mesa 20.3.5 (ACO)
❗ limited,"llvmpipe (LLVM 11.0.1, 256 bits)",CPU,Vulkan,Mesa 20.3.5 (LLVM 11.0.1)
❌,"Radeon RX 570 Series (POLARIS10, DRM 3.40.0, 5.10.0-21-amd64, LLVM 11.0.1)",Unknown,OpenGL,4.6 (Core Profile) Mesa 20.3.5


In [2]:
img = iio.imread("imageio:astronaut.png") @ np.asarray([0.299, 0.587, 0.114])

In [3]:
iw = fpl.ImageWidget(
    [img, np.zeros(img.shape)],
    figure_shape=(2, 1),
    cmap="gray"
)
iw.show(sidecar=True)

RFBOutputContext()

/home/kushal/repos/fastplotlib/fastplotlib/graphics/features/_base.py:18: UserWarning: casting float64 array to float32
  warn(f"casting {array.dtype} array to float32")


In [4]:

block_size = 8

I, J = np.meshgrid(np.arange(block_size), np.arange(block_size), indexing="ij")

# fig = fpl.Figure(shape=(block_size, block_size), size=(1200, 1200), controller_ids="sync")


# def generate_basis():
zigzag = np.array(
    [
        [0, 0],
        [0, 1],
        [1, 0],
        [2, 0],
        [1, 1],
        [0, 2],
        [0, 3],
        [1, 2],
        [2, 1],
        [3, 0],
        [4, 0],
        [3, 1],
        [2, 2],
        [1, 3],
        [0, 4],
        [0, 5],
        [1, 4],
        [2, 3],
        [3, 2],
        [4, 1],
        [5, 0],
        [6, 0],
        [5, 1],
        [4, 2],
        [3, 3],
        [2, 4],
        [1, 5],
        [0, 6],
        [0, 7],
        [1, 6],
        [2, 5],
        [3, 4],
        [4, 3],
        [5, 2],
        [6, 1],
        [7, 0],
        [7, 1],
        [6, 2],
        [5, 3],
        [4, 4],
        [3, 5],
        [2, 6],
        [1, 7],
        [2, 7],
        [3, 6],
        [4, 5],
        [5, 4],
        [6, 3],
        [7, 2],
        [7, 3],
        [6, 4],
        [5, 5],
        [4, 6],
        [3, 7],
        [4, 7],
        [5, 6],
        [6, 5],
        [7, 4],
        [7, 5],
        [6, 6],
        [5, 7],
        [6, 7],
        [7, 6],
        [7, 7],
    ]
)

basis = np.zeros((block_size**2, block_size, block_size))

for i, (u, v) in enumerate(zigzag):
    c = (
        (2 / block_size)
        * np.cos((2 * I + 1) * u * np.pi / (2 * block_size))
        * np.cos((2 * J + 1) * v * np.pi / (2 * block_size))
    )
    c /= np.linalg.norm(c)

    basis[i] = c

In [5]:
rs = iw.managed_graphics[0].add_rectangle_selector(edge_color="w", edge_thickness=1.0, vertex_color="w")

In [6]:
from tqdm import tqdm

In [10]:
basis

array([[[ 0.125     ,  0.125     ,  0.125     , ...,  0.125     ,
          0.125     ,  0.125     ],
        [ 0.125     ,  0.125     ,  0.125     , ...,  0.125     ,
          0.125     ,  0.125     ],
        [ 0.125     ,  0.125     ,  0.125     , ...,  0.125     ,
          0.125     ,  0.125     ],
        ...,
        [ 0.125     ,  0.125     ,  0.125     , ...,  0.125     ,
          0.125     ,  0.125     ],
        [ 0.125     ,  0.125     ,  0.125     , ...,  0.125     ,
          0.125     ,  0.125     ],
        [ 0.125     ,  0.125     ,  0.125     , ...,  0.125     ,
          0.125     ,  0.125     ]],

       [[ 0.17337998,  0.14698445,  0.09821187, ..., -0.09821187,
         -0.14698445, -0.17337998],
        [ 0.17337998,  0.14698445,  0.09821187, ..., -0.09821187,
         -0.14698445, -0.17337998],
        [ 0.17337998,  0.14698445,  0.09821187, ..., -0.09821187,
         -0.14698445, -0.17337998],
        ...,
        [ 0.17337998,  0.14698445,  0.09821187, ..., -

In [12]:
block_indices = list(product(*(range(np.ceil(dim / block_size).astype(int)) for dim in img.shape)))
for u, v in tqdm(block_indices):
    row_start = u * block_size
    row_stop = row_start + block_size
    
    col_start = v * block_size
    col_stop = col_start + block_size
    r_slice, c_slice = slice(row_start, row_stop), slice(col_start, col_stop)
    
    block = img[r_slice, c_slice]
    
    n = 15
    projection = np.zeros((block_size, block_size))
    for i in range(n):
        projection += basis[i] * (basis[i] * block).sum()

    iw.managed_graphics[1].data[r_slice, c_slice] = projection
    iw.managed_graphics[1].reset_vmin_vmax()
    iw.managed_graphics[1].vmin = 0

100%|████████████████████████████████████████████████████████████████████| 4096/4096 [00:05<00:00, 781.32it/s]
